# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Изучение файла

Испортируем библиотеки.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from tqdm import tqdm
import matplotlib.pyplot as plt

Прочитаем данные с файл и запишем в df.

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

Просмотрим общую информацию о файле.

In [3]:
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Согласно документации к данным, в таблице `users_behavior` имеются следующие столбцы:
* `сalls` — количество звонков
* `minutes` — суммарная длительность звонков в минутах
* `messages` — количество sms-сообщений
* `mb_used` — израсходованный интернет-трафик в Мб
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

**Вывод этапа**:
- Проведено чтение данных с файла

## Подготовка данных

Произведем разбиение на целевой признак и остальные.

In [4]:
features = df.drop(['is_ultra'], axis=1)
target = df["is_ultra"]

Выделим тестовую, тренировочную и валидационнную выборки.
25% данных выделим для валидационной выборки.

Выделим тестовую выборку.

In [5]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size = 0.25, random_state = 12345)

Разделим тестовую выборку на тестовую и валидационную (оставим в каждой выборке ао 50%).

In [6]:
features_valid, features_test, target_valid, target_test = train_test_split(
    features_test, target_test, test_size = 0.5, random_state = 12345)

**Вывод этапа**:
- Проведено разделение данных на тестовую, тренировочную и валидационную выборки.

## Исследование моделей

Исследуем Random Forest.

In [7]:
best_model = None
best_accuracy = 0
best_est = 0
best_depth = 0
for est in range(1,50):
    for depth in range(1,25):
        model = RandomForestClassifier(random_state = 12345, n_estimators = est, max_depth = depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        accuracy = accuracy_score(predictions_valid, target_valid)
        if best_accuracy < accuracy:
            best_model = model
            best_depth = depth
            best_est = est
            best_accuracy = accuracy
            
print("Random Forest:", best_accuracy)

Random Forest: 0.8233830845771144


Исследуем Decision Tree.

In [8]:
best_model = None
best_accuracy = 0
best_depth = 0
for depth in range(1,50):
    model2 = DecisionTreeClassifier(random_state = 12345, max_depth = depth)
    model2.fit(features_train,target_train)
    predictions_valid = model2.predict(features_valid)
    accuracy = accuracy_score(predictions_valid, target_valid)
    if best_accuracy < accuracy:
        best_model = model2
        best_accuracy = accuracy
        best_depth = depth
        
print("Decision Tree:", best_accuracy)

Decision Tree: 0.7985074626865671


Исследуем LogisticRegression.

In [9]:
model3 = LogisticRegression(random_state = 12345, max_iter= 1000, solver = 'lbfgs')
model3.fit(features_train,target_train)
predictions_valid= model3.predict(features_valid)
accuracy = accuracy_score(predictions_valid, target_valid)
print("Logistic Regression:", accuracy)

Logistic Regression: 0.7039800995024875


**Вывод этапа**:
Были исследованы модели Random Forest, Desicion Tree и Logistic Regression.

Получены следующие показатели точности:
- Random Forest: 0.8233830845771144
- Decision Tree: 0.7985074626865671
- Logistic Regression: 0.7039800995024875
Из результатов видно, наиболее качественные результаты у Random Forest.

## Проверьте модель на тестовой выборке

Исследуем LogisticRegression.

In [10]:
predictions_test = model3.predict(features_test)
accuracy_test = accuracy_score(predictions_test, target_test)
print("Logistic Regression:", accuracy_test)

Logistic Regression: 0.7039800995024875


Исследуем Random Forest.

In [11]:
predictions_test = model.predict(features_test)
accuracy_test = accuracy_score(predictions_test, target_test)
print("Random Forest:", accuracy_test)

Random Forest: 0.7786069651741293


Исследуем Decision Tree.

In [12]:
predictions_test = model2.predict(features_test)
accuracy_test = accuracy_score(predictions_test, target_test)
print("Decision Tree:", accuracy_test)

Decision Tree: 0.7213930348258707


**Вывод этапа**:
Были исследованы модели Random Forest, Desicion Tree и Logistic Regression.

Получены следующие показатели точности:
- Random Forest: 0.7786069651741293
- Decision Tree: 0.7213930348258707
- Logistic Regression: 0.7039800995024875
Из результатов видно, наиболее качественные результаты у Random Forest, как и в исследовании на тренировочных выборках.

**Общий вывод**:
Итак, полученный архив данных - данные о поведении клиентов, которые уже перешедших на тарифы "Смарт" и "Ультра".

Для исследования было проведено разделение данных на тестовую, тренировочную и валидационную выборки.На этих выборка проверены работы моделей Random Forest, Desicion Tree и Logistic Regression.

Получены следующие показатели точности на валидационной выборке:
- Random Forest: 0.8233830845771144
- Decision Tree: 0.7985074626865671
- Logistic Regression: 0.7039800995024875<br>

На тестовой:
- Random Forest: 0.7786069651741293
- Decision Tree: 0.7213930348258707
- Logistic Regression: 0.7039800995024875
Из результатов видно, и на тестовой и на валидационной выборках наиболее хорошие результаты показывает Random Forest.